## Resonant point found at ~ 180 Hz

In [1]:
# Creating another ODR model to model this resonance observed at a different frequency

mydata = RealData(df150degv2, ov150degv2, sx=DACError, sy=1.2753244632410308)

myodr = ODR(mydata, Lorentzian_model, beta0=[2.6, 183.6884000442675, 1.25])

myoutput = myodr.run()

myoutput.pprint()

NameError: name 'RealData' is not defined

In [ ]:
Iron150degv2 = np.loadtxt('Data/Iron/150degIronDataHone2better')
ov150degv2 = Iron150degv2[:,1]

df150degv2 = []

iv = Iron150degv2[:,0]
print(iv)

for i in iv:
    a= LinearApproximation(myoutput.beta, i)
    df150degv2.append(a)

# Find Peak Frequency 
LinearApproximation(myoutput.beta, 2.850000000000003642e+00) # Input found from Raw Data

In [ ]:
df_fit = np.linspace(min(df150degv2), max(df150degv2), 1000)
ov_fit = Lorentzian(myoutput.beta, df_fit)

In [ ]:
plt.title('Theoretical Model of the Resonance from Data taken using ODR')
plt.xlabel('Driver Frequency (Hz)')
plt.ylabel('Output Voltage (V)')

plt.plot(df150degv2, ov150degv2, '.', label='423.15K', color = 'r')

plt.plot(df_fit, ov_fit, color = 'purple', label='Theoretical Curve')

plt.legend()
plt.show()

## Another Point of Resonance found at ~ 80 Hz

In [ ]:
# Not found at 303.15K

data60degH21 = np.loadtxt('Data/Iron/60degIronDataHone2-1')
df60deg = data60degH21[:,0]
iv60deg= data60degH21[:,1]
ov60deg = data60degH21[:,2]

data90degH21 = np.loadtxt('Data/Iron/90degIronDataHone2-1')
df90deg = data90degH21[:,0]
iv90deg= data90degH21[:,1]
ov90deg = data90degH21[:,2]

data120degH21 = np.loadtxt('Data/Iron/120degIronDataHone2-1')
df120deg = data120degH21[:,0]
iv120deg= data120degH21[:,1]
ov120deg = data120degH21[:,2]

data150degH21 = np.loadtxt('Data/Iron/150degIronDataHone3-1')
iv150deg = data150degH21[:,0]
ov150deg= data150degH21[:,1]

In [ ]:
#  Creating a list the same size as the x vlaues as they all have the same errors

dferror = 0.75 # consistent for all measurements

df60degerr = [dferror] * len(df60deg)
df90degerr = [dferror] * len(df90deg)
df120degerr = [dferror] * len(df120deg)
df150degerr = [dferror] * len(df150deg)

In [ ]:
# plt.plot(df30deg, ov30deg, '.', label = '303.15K')
# plt.plot(df60deg, ov60deg, '.', label = '333.15K')
# plt.plot(df90deg, ov90deg, '.', label='363.15K')
# plt.plot(df120deg, ov120deg, '.', label='393.15K')
# plt.plot(df150deg, ov150deg, '.', label='422.15K')

In [ ]:
# Creating a for loop to sweep through and gather the parameters and errors for all temperatures quickly

# Putting all of my data into a list for the x and y axis
IronData_x_axis = [df60deg, df90deg, df120deg, df150deg]
IronData_y_axis = [ov60deg, ov90deg, ov120deg, ov150deg]
IronData_errors = [df60degerr, df90degerr, df120degerr, df150degerr]
EstimatesofFMHW = [2.25, 0.75, 0.5, 3.75, 0.6] # Estimated using the individual plots of the data above
EstimatesofAmplitudes = [0.25, 0.55, 0.68, 3, 3] 
resonantfreqs = [2.715400000000000205e+02, 2.710199999999999818e+02, 2.693299999999999841e+02, 2.681100000000000136e+02, 2.674700000000000273e+02] # Gottem from Raw data 

w_fits = []
y_fits = []
w0 = []
deltaw = []
sw0 = []
sdeltaw = []
sdeltaww0 = []

for i, j, k, q, p, m in zip(IronData_x_axis, IronData_y_axis, IronData_errors, EstimatesofFMHW, EstimatesofAmplitudes, resonantfreqs):
    # Inputting all experimental data
    w = np.array(i) # Driver Frequencies
    y = np.array(j) # Output Voltages
    sw = np.array(k) # Consistent Fluctuation on frequency measurement
    sy = np.array(OVError) # Error on ADC in NI USB 6008

    data = RealData(w ,y, sx=sw, sy=sy) # Creating the RealData object as specified on the Scipy.ODR webpage with x, y values and their uncertainties.
    # Not including covariances here as the variables are independent.

    # Must guess initial parameters for the parameters in the vector B; A (amplitude), w0 (resonant frequency) and deltaw (Full width half maximum of graph)
    Initial_Estimate = [p, m, q]

    # Instantiating the Blueprint ODR with my experimental data
    myodr = ODR(data, Lorentzian_model, beta0=Initial_Estimate)

    # Running the fit
    output = myodr.run()

    # Appending the values of w0 and deltaw for later calculations:
    w0.append(output.beta[1])

    deltaw.append(output.beta[2])

    # Appending the errors of the parameters above
    sw0.append(output.sd_beta[1])
    sdeltaw.append(output.sd_beta[2])

    #Appending the Covariance of deltaw and w0 for later calculations
    sdeltaww0.append(output.cov_beta[1, 2])

    # Printing a summary of the fit results:
    #output.pprint()

    # Extract the optimal parameters:
    popt = output.beta # Gives optimized parameters A, w0 and deltaw
    perr = output.sd_beta # Gives the Standard deviation of the parameters (errors)

    w_fit = np.linspace(min(w), max(w), 1000)
    y_fit = Lorentzian(popt, w_fit)

    w_fits.append(w_fit)
    y_fits.append(y_fit)

In [ ]:
plt.plot(df60deg, ov60deg, '.', label = '333.15K', color = 'orange')

#plt.plot(w_fits[1], y_fits[1], color = 'orange')

plt.legend()
plt.show()

In [ ]:
plt.plot(df90deg, ov90deg, '.', label='363.15K', color = 'g')

#plt.plot(w_fits[2], y_fits[2], color = 'g')

plt.legend()
plt.show()

In [ ]:
plt.plot(df120deg, ov120deg, '.', label='393.15K', color = 'r')

#plt.plot(w_fits[3], y_fits[3], color = 'r')

plt.legend()
plt.show()

In [ ]:
ov150deg = data150degH21[:,1]

df150deg = []

iv = data150degH21[:,0]
print(iv)

for i in iv:
    a= LinearApproximation(myoutput.beta, i)
    df150deg.append(a)

print(df150deg)

In [ ]:
plt.plot(df150deg, ov150deg, '.', label='423.15K', color = 'cyan')

#plt.plot(w_fits[4], y_fits[4], color = 'r')

plt.legend()
plt.show()